# Video Game Sales Analysis with Pandas

This notebook contains a comprehensive analysis of video game sales data spanning multiple decades, platforms, and genres.

In [36]:
# Part 1: Data Loading and Cleaning

## Task 1.1: Load the Datasets


In [37]:
import pandas as pd
import numpy as np

# Loading the datasets
games_df = pd.read_csv('video_games.csv')
ratings_df = pd.read_csv('game_ratings.csv')

# Quick look at what we're working with
print("=== Video Games Dataset ===")
print(games_df.head())
print(f"\nShape: {games_df.shape}")
print(f"\nColumns: {list(games_df.columns)}")

print("\n" + "="*50)
print("\n=== Game Ratings Dataset ===")
print(ratings_df.head())
print(f"\nShape: {ratings_df.shape}")
print(f"\nColumns: {list(ratings_df.columns)}")


=== Video Games Dataset ===
           Name Platform    Year      Genre             Publisher  NA_Sales  \
0  Championship       DS  2017.0     Action                Capcom      0.04   
1         Quest     XOne  2013.0  Adventure             Microsoft      0.18   
2      Warriors       DS  2015.0     Racing            Activision      1.08   
3    Dark Quest      PS4  1985.0   Fighting             505 Games      0.14   
4     Adventure       PC  2005.0  Adventure  Take-Two Interactive      0.02   

   EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0      0.03      0.01          0.0          0.08  
1      0.10      0.03         -0.0          0.31  
2      0.73      0.49         -0.0          2.30  
3      0.09      0.02          0.0          0.25  
4      0.02      0.00         -0.0          0.05  

Shape: (810, 10)

Columns: ['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']


=== Game Ratings Dataset ===
            Na

In [38]:
# Getting info about data types and structure
print("=== Games Dataset Info ===")
games_df.info()
print("\n" + "="*60)
print("\n=== Ratings Dataset Info ===")
ratings_df.info()


=== Games Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810 entries, 0 to 809
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          810 non-null    object 
 1   Platform      810 non-null    object 
 2   Year          770 non-null    float64
 3   Genre         810 non-null    object 
 4   Publisher     786 non-null    object 
 5   NA_Sales      810 non-null    float64
 6   EU_Sales      810 non-null    float64
 7   JP_Sales      810 non-null    float64
 8   Other_Sales   810 non-null    float64
 9   Global_Sales  810 non-null    float64
dtypes: float64(6), object(4)
memory usage: 63.4+ KB


=== Ratings Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          494 non-null    object 
 1   Platform      494 non-null    

In [39]:
# Checking for missing values
print("=== Missing Values in Games Dataset ===")
missing_games = games_df.isnull().sum()
print(missing_games[missing_games > 0])

print("\n=== Missing Values in Ratings Dataset ===")
missing_ratings = ratings_df.isnull().sum()
print(missing_ratings[missing_ratings > 0])


=== Missing Values in Games Dataset ===
Year         40
Publisher    24
dtype: int64

=== Missing Values in Ratings Dataset ===
Critic_Score    39
Rating          24
dtype: int64


In [40]:
# Summary statistics for the games dataset
games_df.describe()


,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,770.000000,810.000000,810.000000,810.000000,810.0,810.000000
mean,2009.725974,0.870926,0.600457,0.175037,0.0,1.648524
std,8.511232,2.907225,1.943002,0.733376,0.0,5.528638
min,1985.000000,0.000000,0.000000,0.000000,-0.0,0.000000
25%,2005.000000,0.070000,0.050000,0.010000,0.0,0.130000
50%,2012.000000,0.190000,0.130000,0.030000,0.0,0.360000
75%,2016.000000,0.597500,0.440000,0.100000,0.0,1.147500
max,2020.000000,45.860000,29.090000,12.500000,0.0,85.880000


In [41]:
# Checking for duplicates
games_duplicates = games_df.duplicated().sum()
ratings_duplicates = ratings_df.duplicated().sum()

print(f"Duplicate rows in games dataset: {games_duplicates}")
print(f"Duplicate rows in ratings dataset: {ratings_duplicates}")


Duplicate rows in games dataset: 10
Duplicate rows in ratings dataset: 8


### Answers to Initial Exploration Questions:

1. **How many missing values are in the Year column?**
   - Answer: 40 missing values in the Year column

2. **Which columns have missing values in the ratings dataset?**
   - Answer: Critic_Score (39 missing values) and Rating (24 missing values)

3. **Are there any duplicate rows?**
   - Answer: Yes, there are 10 duplicate rows in the games dataset and 8 duplicate rows in the ratings dataset


## Task 1.3: Data Cleaning

### Cleaning the Games Dataset


In [42]:
# Store original shape for reporting
original_games_shape = games_df.shape
print(f"Original games dataset shape: {original_games_shape}")

# Remove rows where Year is missing
games_df = games_df.dropna(subset=['Year'])
print(f"After removing missing Year: {games_df.shape}")

# Convert Year to integer
games_df['Year'] = games_df['Year'].astype(int)

# Remove rows where Publisher is missing
games_df = games_df.dropna(subset=['Publisher'])
print(f"After removing missing Publisher: {games_df.shape}")

# Remove duplicate rows
games_df = games_df.drop_duplicates()
print(f"After removing duplicates: {games_df.shape}")


Original games dataset shape: (810, 10)
After removing missing Year: (770, 10)
After removing missing Publisher: (748, 10)
After removing duplicates: (738, 10)


In [43]:
# Verify that Global_Sales equals sum of regional sales
# Let's check if there are any discrepancies
regional_sum = games_df['NA_Sales'] + games_df['EU_Sales'] + games_df['JP_Sales'] + games_df['Other_Sales']
difference = abs(games_df['Global_Sales'] - regional_sum)

# Check how many rows have significant differences (more than 0.01 million)
significant_diff = (difference > 0.01).sum()
print(f"Rows with significant difference (>0.01M): {significant_diff}")

# For rows with differences, let's recalculate Global_Sales
games_df['Global_Sales'] = regional_sum

print(f"Global_Sales column has been recalculated to match regional sum")


Rows with significant difference (>0.01M): 161
Global_Sales column has been recalculated to match regional sum


### Cleaning the Ratings Dataset


In [44]:
# Store original shape for reporting
original_ratings_shape = ratings_df.shape
print(f"Original ratings dataset shape: {original_ratings_shape}")

# Handle User_Score column - replace 'tbd' with NaN
ratings_df['User_Score'] = ratings_df['User_Score'].replace('tbd', np.nan)

# Convert to numeric (this will also convert any remaining strings to NaN)
ratings_df['User_Score'] = pd.to_numeric(ratings_df['User_Score'], errors='coerce')

# Remove rows where both Critic_Score and User_Score are missing
ratings_df = ratings_df.dropna(subset=['Critic_Score', 'User_Score'], how='all')
print(f"After removing rows with both scores missing: {ratings_df.shape}")

# Remove duplicate rows
ratings_df = ratings_df.drop_duplicates()
print(f"After removing duplicates: {ratings_df.shape}")


Original ratings dataset shape: (494, 5)
After removing rows with both scores missing: (493, 5)
After removing duplicates: (485, 5)


In [45]:
# Reporting cleaning results
games_removed = original_games_shape[0] - games_df.shape[0]
ratings_removed = original_ratings_shape[0] - ratings_df.shape[0]

print("=== Cleaning Summary ===")
print(f"Games dataset:")
print(f"  - Rows removed: {games_removed}")
print(f"  - Original shape: {original_games_shape}")
print(f"  - New shape: {games_df.shape}")

print(f"\nRatings dataset:")
print(f"  - Rows removed: {ratings_removed}")
print(f"  - Original shape: {original_ratings_shape}")
print(f"  - New shape: {ratings_df.shape}")


=== Cleaning Summary ===
Games dataset:
  - Rows removed: 72
  - Original shape: (810, 10)
  - New shape: (738, 10)

Ratings dataset:
  - Rows removed: 9
  - Original shape: (494, 5)
  - New shape: (485, 5)


# Part 2: Exploratory Data Analysis

## Task 2.1: Top Games Analysis


In [46]:
# Top 10 best-selling games globally
top_10_games = games_df.nlargest(10, 'Global_Sales')[['Name', 'Platform', 'Year', 'Global_Sales']]
print("Top 10 Best-Selling Games Globally:")
print(top_10_games.to_string(index=False))


Top 10 Best-Selling Games Globally:
           Name Platform  Year  Global_Sales
      Adventure      PS4  2014         83.24
      Adventure      PS4  2015         46.32
         Sports      3DS  2015         41.81
     Dark Quest       DS  2014         35.29
         Strike      PS3  2014         25.26
      Simulator     XOne  2014         22.84
Super Simulator      PS4  2008         21.64
          Force      PS2  2016         21.38
       Warriors     X360  2014         17.92
  Shadow Racing       DS  2015         17.19


In [47]:
# Top 5 publishers by total global sales
publisher_sales = games_df.groupby('Publisher')['Global_Sales'].sum().sort_values(ascending=False)
top_5_publishers = publisher_sales.head(5)

print("Top 5 Publishers by Total Global Sales:")
for publisher, sales in top_5_publishers.items():
    print(f"{publisher}: {sales:.2f} million units")


Top 5 Publishers by Total Global Sales:
Electronic Arts: 167.29 million units
Ubisoft: 159.15 million units
Sony: 144.06 million units
Nintendo: 112.08 million units
Take-Two Interactive: 78.22 million units


### Answers:

1. **What is the best-selling game of all time?**
   - Answer: "Adventure" for PS4 released in 2014, with 83.24 million units sold globally

2. **Which publisher has the highest total sales?**
   - Answer: Electronic Arts has the highest total sales with 167.29 million units, followed by Ubisoft (159.15M), Sony (144.06M), Nintendo (112.08M), and Take-Two Interactive (78.22M)

## Task 2.2: Platform Analysis


In [48]:
# Count games per platform
games_per_platform = games_df['Platform'].value_counts()
print("Number of games per platform:")
print(games_per_platform.head(10))


Number of games per platform:
Platform
PS3       87
PS4       82
PS2       75
Wii       73
X360      69
DS        61
PC        58
XOne      55
Xbox      45
Switch    43
Name: count, dtype: int64


In [49]:
# Total sales by platform
platform_sales = games_df.groupby('Platform')['Global_Sales'].sum().sort_values(ascending=False)
print("Total sales by platform (top 10):")
print(platform_sales.head(10))


Total sales by platform (top 10):
Platform
PS4       252.07
DS        172.96
PS2       125.10
Wii       121.65
PS3       121.44
X360      112.57
3DS        73.83
XOne       47.83
PC         29.48
Switch     27.14
Name: Global_Sales, dtype: float64


In [50]:
# Average sales per game for each platform
platform_avg_sales = games_df.groupby('Platform')['Global_Sales'].mean().sort_values(ascending=False)
print("Average sales per game by platform (top 10):")
print(platform_avg_sales.head(10))


Average sales per game by platform (top 10):
Platform
PS4     3.074024
3DS     2.839615
DS      2.835410
PS      1.686667
PS2     1.668000
Wii     1.666438
X360    1.631449
PS3     1.395862
GBA     1.180000
PSP     0.926842
Name: Global_Sales, dtype: float64


### Answers:

1. **Which platform has the most games?**
   - Answer: PS3 has the most games with 87 titles, followed by PS4 (82), PS2 (75), and Wii (73)

2. **Which platform has the highest total sales?**
   - Answer: PS4 has the highest total sales with 252.07 million units, significantly ahead of DS (172.96M) and PS2 (125.10M)

3. **Which platform has the highest average sales per game?**
   - Answer: PS4 has the highest average sales per game at 3.07 million units per game, followed by 3DS (2.84M) and DS (2.84M)


## Task 2.3: Genre Analysis


In [51]:
# Total sales by genre
genre_total_sales = games_df.groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False)

# Number of games per genre
genre_game_count = games_df['Genre'].value_counts()

# Average sales per game by genre
genre_avg_sales = games_df.groupby('Genre')['Global_Sales'].mean().sort_values(ascending=False)

# Create summary table
genre_summary = pd.DataFrame({
    'Total_Sales': genre_total_sales,
    'Number_of_Games': genre_game_count,
    'Avg_Sales_per_Game': genre_avg_sales
})

print("Genre Analysis Summary:")
print(genre_summary)


Genre Analysis Summary:
              Total_Sales  Number_of_Games  Avg_Sales_per_Game
Genre                                                         
Action             220.48              140            1.574857
Adventure           39.45               58            0.680172
Fighting            64.30               47            1.368085
Misc                 9.87               14            0.705000
Platform            88.42               57            1.551228
Puzzle              18.29               32            0.571562
Racing              80.37               59            1.362203
Role-Playing        85.50               74            1.155405
Shooter            151.97               90            1.688556
Simulation          90.77               20            4.538500
Sports             282.53              116            2.435603
Strategy            22.57               31            0.728065


### Answers:

1. **What is the most popular genre by total sales?**
   - Answer: Sports is the most popular genre by total sales with 282.53 million units, followed by Action (220.48M) and Shooter (151.97M)

2. **Which genre has the highest average sales per game?**
   - Answer: Simulation has the highest average sales per game at 4.54 million units per game, despite having only 20 games. Sports comes second with 2.44M average sales per game

3. **Do you notice any interesting patterns?**
   - Answer: There's an interesting inverse relationship between number of games and average sales for some genres. Simulation has very few games (20) but the highest average sales (4.54M), suggesting quality over quantity. Sports has both high total sales (282.53M) and high average sales (2.44M), making it a consistently strong performer. Action has the most games (140) and second-highest total sales, but lower average sales (1.57M), indicating a volume-based strategy.


## Task 2.4: Temporal Analysis


In [52]:
# Year range
min_year = games_df['Year'].min()
max_year = games_df['Year'].max()
print(f"Year range: {min_year} to {max_year}")

# Games released per year
games_per_year = games_df['Year'].value_counts().sort_index()

# Total sales per year
sales_per_year = games_df.groupby('Year')['Global_Sales'].sum().sort_values(ascending=False)

print(f"\nYear with highest sales: {sales_per_year.index[0]} ({sales_per_year.iloc[0]:.2f} million)")
print(f"Year with most game releases: {games_per_year.idxmax()} ({games_per_year.max()} games)")


Year range: 1985 to 2020

Year with highest sales: 2014 (240.64 million)
Year with most game releases: 2015 (55 games)


In [53]:
# Let's look at the trends over time
yearly_stats = pd.DataFrame({
    'Games_Released': games_per_year,
    'Total_Sales': games_df.groupby('Year')['Global_Sales'].sum()
})

print("Yearly Statistics (showing first and last 10 years):")
print(yearly_stats.head(10))
print("\n...\n")
print(yearly_stats.tail(10))


Yearly Statistics (showing first and last 10 years):
      Games_Released  Total_Sales
Year                             
1985               8         0.76
1986               5         0.60
1987               3         0.32
1988               6         2.94
1989               4         1.70
1990               2         1.31
1991               2         6.40
1992               6         4.23
1993               4         0.52
1994               5         1.73

...

      Games_Released  Total_Sales
Year                             
2011              27        52.20
2012              32        22.39
2013              23        19.19
2014              39       240.64
2015              55       157.20
2016              50        79.24
2017              45        69.09
2018              47        69.68
2019              38        60.44
2020              49        67.95


### Answers:

1. **What year had the highest total sales?**
   - Answer: 2014 had the highest total sales with 240.64 million units, which is significantly higher than other years

2. **What year had the most game releases?**
   - Answer: 2015 had the most game releases with 55 games, followed by 2016 (50 games) and 2018 (47 games)

3. **What trends do you observe over time?**
   - Answer: The gaming industry shows clear growth trends over time. The 2010s decade dominates with 385 games released and 802.12M total sales. There's a significant spike in 2014 with 240.64M sales despite only 39 games released, suggesting blockbuster titles that year. The number of games released has generally increased from the 1980s (26 games) to the 2010s (385 games), showing market expansion. However, average sales per game peaked in the 2010s at 2.08M, indicating both market growth and the success of major franchises.


# Part 3: Advanced Analysis

## Task 3.1: Regional Sales Analysis


In [54]:
# Calculate percentage of global sales from each region
total_global = games_df['Global_Sales'].sum()
na_percentage = (games_df['NA_Sales'].sum() / total_global) * 100
eu_percentage = (games_df['EU_Sales'].sum() / total_global) * 100
jp_percentage = (games_df['JP_Sales'].sum() / total_global) * 100
other_percentage = (games_df['Other_Sales'].sum() / total_global) * 100

print("Regional Sales as Percentage of Global Sales:")
print(f"North America: {na_percentage:.2f}%")
print(f"Europe: {eu_percentage:.2f}%")
print(f"Japan: {jp_percentage:.2f}%")
print(f"Other Regions: {other_percentage:.2f}%")


Regional Sales as Percentage of Global Sales:
North America: 52.69%
Europe: 36.80%
Japan: 10.51%
Other Regions: 0.00%


In [55]:
# Top 5 games in North America
top_5_na = games_df.nlargest(5, 'NA_Sales')[['Name', 'Platform', 'NA_Sales']]
print("Top 5 Games in North America:")
print(top_5_na.to_string(index=False))

# Top 5 games in Japan
top_5_jp = games_df.nlargest(5, 'JP_Sales')[['Name', 'Platform', 'JP_Sales']]
print("\nTop 5 Games in Japan:")
print(top_5_jp.to_string(index=False))


Top 5 Games in North America:
      Name Platform  NA_Sales
 Adventure      PS4     45.75
 Adventure      PS4     22.12
    Sports      3DS     18.52
Dark Quest       DS     16.04
 Simulator     XOne     13.63

Top 5 Games in Japan:
      Name Platform  JP_Sales
 Adventure      PS4      8.40
    Sports      3DS      8.01
Dark Quest       DS      7.51
 Adventure      PS4      4.32
     Force      PS2      3.79


In [56]:
# Compare: which games appear in both top 5 lists?
na_games = set(top_5_na['Name'])
jp_games = set(top_5_jp['Name'])
common_games = na_games.intersection(jp_games)

print(f"Games appearing in both top 5 lists: {len(common_games)}")
if common_games:
    print("Common games:", list(common_games))
else:
    print("No games appear in both top 5 lists")


Games appearing in both top 5 lists: 3
Common games: ['Dark Quest', 'Sports', 'Adventure']


In [57]:
# Most popular genre in each region
na_genre = games_df.groupby('Genre')['NA_Sales'].sum().idxmax()
eu_genre = games_df.groupby('Genre')['EU_Sales'].sum().idxmax()
jp_genre = games_df.groupby('Genre')['JP_Sales'].sum().idxmax()
other_genre = games_df.groupby('Genre')['Other_Sales'].sum().idxmax()

print("Most Popular Genre by Region:")
print(f"North America: {na_genre}")
print(f"Europe: {eu_genre}")
print(f"Japan: {jp_genre}")
print(f"Other Regions: {other_genre}")


Most Popular Genre by Region:
North America: Sports
Europe: Sports
Japan: Sports
Other Regions: Action


### Answers:

1. **Which region contributes the most to global sales?**
   - Answer: North America contributes the most to global sales at 52.69%, followed by Europe (36.80%) and Japan (10.51%). Other regions contribute 0.00% in this dataset.

2. **Are there significant differences in game preferences between NA and Japan?**
   - Answer: Yes, there are notable differences. While 3 games appear in both top 5 lists (Adventure, Sports, and Dark Quest), the sales figures show regional preferences. For example, "Adventure" for PS4 sold 45.75M in NA but only 8.40M in Japan. Interestingly, Sports is the most popular genre in both regions, suggesting some universal appeal. However, the top games in each region have different sales distributions, with NA games generally having much higher absolute sales numbers, reflecting the larger market size.


## Task 3.2: Publisher Deep Dive


In [58]:
# Get top 5 publishers
top_5_publisher_names = publisher_sales.head(5).index.tolist()

# Create summary for each publisher
publisher_summary_list = []

for pub in top_5_publisher_names:
    pub_data = games_df[games_df['Publisher'] == pub]
    
    total_games = len(pub_data)
    avg_sales = pub_data['Global_Sales'].mean()
    best_genre = pub_data.groupby('Genre')['Global_Sales'].sum().idxmax()
    best_game = pub_data.loc[pub_data['Global_Sales'].idxmax(), 'Name']
    best_game_sales = pub_data['Global_Sales'].max()
    
    publisher_summary_list.append({
        'Publisher': pub,
        'Total_Games': total_games,
        'Avg_Sales_per_Game': round(avg_sales, 2),
        'Most_Successful_Genre': best_genre,
        'Best_Selling_Game': best_game,
        'Best_Game_Sales': round(best_game_sales, 2)
    })

publisher_summary_df = pd.DataFrame(publisher_summary_list)
print("Top 5 Publishers Deep Dive:")
print(publisher_summary_df.to_string(index=False))


Top 5 Publishers Deep Dive:
           Publisher  Total_Games  Avg_Sales_per_Game Most_Successful_Genre Best_Selling_Game  Best_Game_Sales
     Electronic Arts           78                2.14                Sports         Adventure            46.32
             Ubisoft           51                3.12            Simulation         Adventure            83.24
                Sony           71                2.03                Sports            Sports            41.81
            Nintendo           94                1.19                Action     Shadow Racing            17.19
Take-Two Interactive           40                1.96               Shooter            Strike            25.26


## Task 3.3: Merging Datasets and Rating Analysis


In [59]:
# Merge the datasets on Name and Platform
merged_df = games_df.merge(ratings_df, on=['Name', 'Platform'], how='inner')

print(f"Games with rating information: {len(merged_df)}")
print(f"Total games in dataset: {len(games_df)}")
print(f"Percentage with ratings: {(len(merged_df) / len(games_df) * 100):.2f}%")


Games with rating information: 1540
Total games in dataset: 738
Percentage with ratings: 208.67%


In [60]:
# Calculate correlations (only for rows with both scores)
critic_correlation = merged_df[['Critic_Score', 'Global_Sales']].corr().iloc[0, 1]
user_correlation = merged_df[['User_Score', 'Global_Sales']].corr().iloc[0, 1]

print(f"Correlation between Critic_Score and Global_Sales: {critic_correlation:.4f}")
print(f"Correlation between User_Score and Global_Sales: {user_correlation:.4f}")


Correlation between Critic_Score and Global_Sales: 0.1467
Correlation between User_Score and Global_Sales: 0.0433


In [61]:
# Games with high critic scores but low sales (hidden gems)
high_critic_low_sales = merged_df[(merged_df['Critic_Score'] > 90) & (merged_df['Global_Sales'] < 1)]
print("Hidden Gems (High Critic Score >90, Low Sales <1M):")
if len(high_critic_low_sales) > 0:
    print(high_critic_low_sales[['Name', 'Platform', 'Critic_Score', 'Global_Sales']].to_string(index=False))
else:
    print("No games found matching these criteria")


Hidden Gems (High Critic Score >90, Low Sales <1M):
        Name Platform  Critic_Score  Global_Sales
   Simulator      Wii         100.0          0.03
       Force      PS2         100.0          0.09
    Warriors     X360         100.0          0.84
Super Battle     X360         100.0          0.78
Championship      3DS         100.0          0.17
   Adventure      PS4         100.0          0.59
       Quest      PS2         100.0          0.06
      Racing      Wii          95.0          0.23
      Racing      Wii         100.0          0.23
   Adventure      PS4         100.0          0.26
  Tournament     X360          97.0          0.21
       Force      PS2         100.0          0.12
       Quest       DS         100.0          0.21
  Tournament     X360          97.0          0.19
       Quest     X360         100.0          0.08
   Simulator     XOne         100.0          0.99
   Adventure      PS4         100.0          0.54
       Force      PS3          94.0          0.0

In [62]:
# Games with low critic scores but high sales (overhyped)
low_critic_high_sales = merged_df[(merged_df['Critic_Score'] < 50) & (merged_df['Global_Sales'] > 5)]
print("Overhyped Games (Low Critic Score <50, High Sales >5M):")
if len(low_critic_high_sales) > 0:
    print(low_critic_high_sales[['Name', 'Platform', 'Critic_Score', 'Global_Sales']].to_string(index=False))
else:
    print("No games found matching these criteria")


Overhyped Games (Low Critic Score <50, High Sales >5M):
        Name Platform  Critic_Score  Global_Sales
   Adventure      PS4          42.0         16.97
   Simulator      Wii          39.0         10.97
   Simulator      Wii          40.0         10.97
   Simulator      Wii          44.0         10.97
   Simulator      Wii          36.0         10.97
      Racing      Wii          41.0          9.45
   Adventure      PS4          42.0         46.32
      Heroes      PS3          40.0          7.22
    Warriors      Wii          45.0          5.33
  Tournament     X360          28.0          5.79
  Tournament     X360          20.0          5.79
  Tournament     X360          26.0          5.79
      Sports     X360          41.0          8.77
       Quest       DS          47.0          7.22
       Quest       DS          46.0          7.22
      Strike      PS3          31.0         25.26
      Battle      PS2          35.0          6.08
      Battle      PS2          34.0         

### Answers:

1. **Do critic scores correlate with sales? How strongly?**
   - Answer: Critic scores show a weak positive correlation with sales (0.1467). This suggests that while there is some relationship between critical acclaim and commercial success, it's not very strong. Many other factors like marketing, brand recognition, and timing likely play more significant roles.

2. **Do user scores correlate with sales?**
   - Answer: User scores show an even weaker correlation with sales (0.0433), which is almost negligible. This indicates that user ratings have very little predictive power for sales performance, possibly because user scores may reflect post-purchase satisfaction rather than purchase decisions.

3. **Can you find any "hidden gems" or "overhyped games"?**
   - Answer: Yes, there are many examples of both. Hidden gems include games like "Simulator" for Wii (100.0 critic score, 0.03M sales), "Force" for PS2 (100.0 critic score, 0.09M sales), and "Warriors" for X360 (100.0 critic score, 0.84M sales). Overhyped games include "Adventure" for PS4 (42.0 critic score, 83.24M sales), "Strike" for PS3 (31.0 critic score, 25.26M sales), and "Simulator" for Wii (39.0 critic score, 10.97M sales). This shows that critical reception doesn't always align with commercial success.


## Task 3.4: Create New Features


In [63]:
# Create Decade column
games_df['Decade'] = (games_df['Year'] // 10) * 10
games_df['Decade'] = games_df['Decade'].astype(str) + 's'

# Verify the decade column
print("Decade distribution:")
print(games_df['Decade'].value_counts().sort_index())


Decade distribution:
Decade
1980s     26
1990s     83
2000s    195
2010s    385
2020s     49
Name: count, dtype: int64


In [64]:
# Create Sales_Category column
def categorize_sales(sales):
    if sales > 5:
        return 'Blockbuster'
    elif sales > 1:
        return 'Hit'
    else:
        return 'Modest'

games_df['Sales_Category'] = games_df['Global_Sales'].apply(categorize_sales)

# Percentage of games in each category
category_percentages = (games_df['Sales_Category'].value_counts(normalize=True) * 100).round(2)
print("Percentage of games in each sales category:")
print(category_percentages)


Percentage of games in each sales category:
Sales_Category
Modest         72.90
Hit            20.33
Blockbuster     6.78
Name: proportion, dtype: float64


In [65]:
# Which genre has the highest percentage of Blockbuster games?
blockbuster_by_genre = games_df[games_df['Sales_Category'] == 'Blockbuster'].groupby('Genre').size()
total_by_genre = games_df.groupby('Genre').size()
blockbuster_percentage = (blockbuster_by_genre / total_by_genre * 100).sort_values(ascending=False)

print("Percentage of Blockbuster games by genre:")
print(blockbuster_percentage)


Percentage of Blockbuster games by genre:
Genre
Shooter         10.000000
Sports           9.482759
Action           8.571429
Fighting         8.510638
Role-Playing     6.756757
Strategy         6.451613
Racing           5.084746
Simulation       5.000000
Platform         3.508772
Puzzle           3.125000
Adventure             NaN
Misc                  NaN
dtype: float64


# Part 4: Insights and Open Exploration

## Task 4.1: Answer Analytical Questions

### Platform Wars: Which gaming platform generation "won"?


In [66]:
# Let's compare different console generations
# PS2 vs Xbox vs GameCube era (around 2000-2006)
gen6_platforms = ['PS2', 'Xbox', 'GC']
gen6_sales = games_df[games_df['Platform'].isin(gen6_platforms)].groupby('Platform')['Global_Sales'].sum()

# PS3 vs Xbox360 vs Wii era (around 2006-2013)
gen7_platforms = ['PS3', 'Xbox360', 'Wii']
gen7_sales = games_df[games_df['Platform'].isin(gen7_platforms)].groupby('Platform')['Global_Sales'].sum()

# PS4 vs XboxOne era (around 2013-2020)
gen8_platforms = ['PS4', 'XOne']
gen8_sales = games_df[games_df['Platform'].isin(gen8_platforms)].groupby('Platform')['Global_Sales'].sum()

print("Generation 6 (PS2/Xbox/GameCube):")
print(gen6_sales.sort_values(ascending=False))
print(f"\nWinner: {gen6_sales.idxmax()} with {gen6_sales.max():.2f}M sales")

print("\nGeneration 7 (PS3/Xbox360/Wii):")
print(gen7_sales.sort_values(ascending=False))
print(f"\nWinner: {gen7_sales.idxmax()} with {gen7_sales.max():.2f}M sales")

print("\nGeneration 8 (PS4/XboxOne):")
print(gen8_sales.sort_values(ascending=False))
print(f"\nWinner: {gen8_sales.idxmax()} with {gen8_sales.max():.2f}M sales")


Generation 6 (PS2/Xbox/GameCube):
Platform
PS2     125.10
Xbox     16.46
GC        0.30
Name: Global_Sales, dtype: float64

Winner: PS2 with 125.10M sales

Generation 7 (PS3/Xbox360/Wii):
Platform
Wii    121.65
PS3    121.44
Name: Global_Sales, dtype: float64

Winner: Wii with 121.65M sales

Generation 8 (PS4/XboxOne):
Platform
PS4     252.07
XOne     47.83
Name: Global_Sales, dtype: float64

Winner: PS4 with 252.07M sales


**Analysis:** Based on the data above, we can see which platform won each generation. For Generation 6, the PS2 dominated with significantly higher sales than Xbox and GameCube. In Generation 7, the Wii emerged as the winner, likely due to its innovative motion controls and broader appeal to casual gamers. Generation 8 shows the PS4 leading over Xbox One. Overall, Sony's PlayStation platforms have been consistently strong across multiple generations, while Nintendo's Wii achieved remarkable success by targeting a different market segment.


### Publisher Strategy: Compare Nintendo and EA


In [67]:
# Compare Nintendo and EA
nintendo_data = games_df[games_df['Publisher'] == 'Nintendo']
ea_data = games_df[games_df['Publisher'] == 'Electronic Arts']

print("=== Nintendo vs EA Comparison ===")
print(f"\nNintendo:")
print(f"  Number of games: {len(nintendo_data)}")
print(f"  Average sales per game: {nintendo_data['Global_Sales'].mean():.2f}M")
print(f"  Total sales: {nintendo_data['Global_Sales'].sum():.2f}M")
print(f"  Top genre: {nintendo_data.groupby('Genre')['Global_Sales'].sum().idxmax()}")
print(f"  NA sales %: {(nintendo_data['NA_Sales'].sum() / nintendo_data['Global_Sales'].sum() * 100):.1f}%")
print(f"  JP sales %: {(nintendo_data['JP_Sales'].sum() / nintendo_data['Global_Sales'].sum() * 100):.1f}%")

print(f"\nElectronic Arts:")
print(f"  Number of games: {len(ea_data)}")
print(f"  Average sales per game: {ea_data['Global_Sales'].mean():.2f}M")
print(f"  Total sales: {ea_data['Global_Sales'].sum():.2f}M")
print(f"  Top genre: {ea_data.groupby('Genre')['Global_Sales'].sum().idxmax()}")
print(f"  NA sales %: {(ea_data['NA_Sales'].sum() / ea_data['Global_Sales'].sum() * 100):.1f}%")
print(f"  JP sales %: {(ea_data['JP_Sales'].sum() / ea_data['Global_Sales'].sum() * 100):.1f}%")


=== Nintendo vs EA Comparison ===

Nintendo:
  Number of games: 94
  Average sales per game: 1.19M
  Total sales: 112.08M
  Top genre: Action
  NA sales %: 50.7%
  JP sales %: 12.4%

Electronic Arts:
  Number of games: 78
  Average sales per game: 2.14M
  Total sales: 167.29M
  Top genre: Sports
  NA sales %: 52.0%
  JP sales %: 10.6%


**Analysis:** The data reveals interesting strategic differences between Nintendo and EA. Contrary to common perception, EA actually has higher average sales per game (2.14M) compared to Nintendo (1.19M), despite Nintendo releasing more games (94 vs 78). However, EA focuses heavily on Sports games, while Nintendo's top genre is Action. Both publishers have similar regional distributions, with NA accounting for about 50-52% of sales and Japan around 10-12%. EA's total sales (167.29M) exceed Nintendo's (112.08M) in this dataset, but Nintendo's broader game portfolio suggests a more diverse strategy. EA's success in Sports games aligns with their annual franchise releases, while Nintendo's Action focus reflects their platform and adventure game strengths.


### Industry Evolution: 1980s to 2010s


In [68]:
# Analyze industry evolution by decade
decade_analysis = games_df.groupby('Decade').agg({
    'Name': 'count',  # Number of games
    'Global_Sales': 'sum',  # Total sales
    'Platform': 'nunique',  # Platform diversity
    'Genre': lambda x: x.value_counts().index[0]  # Most popular genre
}).rename(columns={'Name': 'Games_Released', 'Global_Sales': 'Total_Sales', 'Platform': 'Unique_Platforms', 'Genre': 'Top_Genre'})

# Calculate average sales per game per decade
decade_analysis['Avg_Sales_per_Game'] = decade_analysis['Total_Sales'] / decade_analysis['Games_Released']

print("Industry Evolution by Decade:")
print(decade_analysis)


Industry Evolution by Decade:
        Games_Released  Total_Sales  Unique_Platforms Top_Genre  \
Decade                                                            
1980s               26         6.32                12    Action   
1990s               83        56.18                15   Shooter   
2000s              195       221.95                15    Action   
2010s              385       802.12                17    Sports   
2020s               49        67.95                14    Action   

        Avg_Sales_per_Game  
Decade                      
1980s             0.243077  
1990s             0.676867  
2000s             1.138205  
2010s             2.083429  
2020s             1.386735  


**Analysis:** The gaming industry has undergone dramatic transformation from the 1980s to 2010s. The number of games released increased from 26 in the 1980s to 385 in the 2010s - nearly a 15-fold increase. Total sales grew exponentially from 6.32M (1980s) to 802.12M (2010s), representing over 125x growth. Platform diversity expanded from 12 unique platforms in the 1980s to 17 in the 2010s. Interestingly, average sales per game increased significantly over time: 0.24M (1980s) to 2.08M (2010s), suggesting that blockbuster titles became more common and successful. Genre preferences shifted: Action dominated the 1980s and 2000s, Shooter was top in the 1990s, and Sports became dominant in the 2010s. The 2010s represent the industry's peak in both volume and average performance, reflecting market maturity and the success of major franchises.


## Task 4.2: Find Something Interesting

### Question: Are there games that were successful in one region but flopped in others?


In [69]:
# Games with high NA sales relative to global
games_df['NA_Ratio'] = games_df['NA_Sales'] / games_df['Global_Sales']
high_na_ratio = games_df[games_df['NA_Ratio'] > 0.8].nlargest(10, 'NA_Sales')[['Name', 'Platform', 'NA_Sales', 'Global_Sales', 'NA_Ratio']]

print("Games with very high NA sales relative to global (NA-focused):")
print(high_na_ratio.to_string(index=False))

# Games with high JP sales relative to global
games_df['JP_Ratio'] = games_df['JP_Sales'] / games_df['Global_Sales']
high_jp_ratio = games_df[games_df['JP_Ratio'] > 0.8].nlargest(10, 'JP_Sales')[['Name', 'Platform', 'JP_Sales', 'Global_Sales', 'JP_Ratio']]

print("\nGames with very high JP sales relative to global (JP-focused):")
print(high_jp_ratio.to_string(index=False))


Games with very high NA sales relative to global (NA-focused):
       Name Platform  NA_Sales  Global_Sales  NA_Ratio
     Sports     WiiU      0.01          0.01       1.0
    Legends       PC      0.01          0.01       1.0
      Force     WiiU      0.01          0.01       1.0
      Quest       PC      0.01          0.01       1.0
Pro Legends      3DS      0.01          0.01       1.0

Games with very high JP sales relative to global (JP-focused):
Empty DataFrame
Columns: [Name, Platform, JP_Sales, Global_Sales, JP_Ratio]
Index: []


In [70]:
# Let's also look at games that sold well in one region but poorly in others
# Games with high NA sales but low JP sales
na_focused = games_df[(games_df['NA_Sales'] > 2) & (games_df['JP_Sales'] < 0.5)].nlargest(10, 'NA_Sales')
print("\nGames with strong NA sales (>2M) but weak JP sales (<0.5M):")
print(na_focused[['Name', 'Platform', 'NA_Sales', 'JP_Sales', 'Global_Sales']].to_string(index=False))

# Games with high JP sales but low NA sales
jp_focused = games_df[(games_df['JP_Sales'] > 2) & (games_df['NA_Sales'] < 0.5)].nlargest(10, 'JP_Sales')
print("\nGames with strong JP sales (>2M) but weak NA sales (<0.5M):")
print(jp_focused[['Name', 'Platform', 'NA_Sales', 'JP_Sales', 'Global_Sales']].to_string(index=False))



Games with strong NA sales (>2M) but weak JP sales (<0.5M):
             Name Platform  NA_Sales  JP_Sales  Global_Sales
     Turbo Battle      PS4      7.04      0.32         11.31
     Super Battle     X360      6.29      0.30         10.21
           Racing      Wii      5.71      0.40          9.45
           Sports     X360      5.13      0.30          8.77
     Super Strike      PS4      4.24      0.41          7.10
           Battle     X360      4.03      0.47          6.72
       Tournament     X360      3.19      0.30          5.79
Dark Championship     X360      3.14      0.30          5.28
           Strike      PS3      2.88      0.21          4.81
           Heroes   Switch      2.77      0.12          4.47

Games with strong JP sales (>2M) but weak NA sales (<0.5M):
Empty DataFrame
Columns: [Name, Platform, NA_Sales, JP_Sales, Global_Sales]
Index: []


**Findings:** This analysis reveals fascinating regional preferences in gaming. The data shows clear evidence of games that succeed in North America but struggle in Japan. For example, "Turbo Battle" for PS4 sold 7.04M units in NA but only 0.32M in Japan, and "Super Battle" for X360 sold 6.29M in NA versus 0.30M in Japan. Interestingly, no games were found with strong JP sales (>2M) but weak NA sales (<0.5M), suggesting that successful Japanese games tend to have broader appeal, or that the dataset may be skewed toward Western markets. The analysis shows that many action-oriented and shooter games (like those on X360 and PS4) perform well in NA but poorly in Japan, likely due to cultural preferences. This insight is valuable for publishers planning global releases, as it highlights the importance of understanding regional markets rather than assuming universal appeal. The data emphasizes the need for targeted marketing and localization strategies, especially for games targeting specific regional preferences.
